<a href="https://colab.research.google.com/github/enesemretas/mcpath-colab/blob/main/notebooks/mcpath_form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# @title MCPath (use repo's ui.py + readpdb_strict.py)
!pip -q install pyyaml ipywidgets requests  # numpy is already in Colab

import sys, pathlib, shutil, subprocess

REPO_URL = "https://github.com/enesemretas/mcpath-colab.git"
REPO_DIR = pathlib.Path("/content/mcpath-colab")

# Fresh clone
if REPO_DIR.exists():
    shutil.rmtree(REPO_DIR)
subprocess.run(["git", "clone", "--depth", "1", REPO_URL, str(REPO_DIR)], check=True)

# Import & launch
import importlib, subprocess, sys
subprocess.run(["git", "-C", "/content/mcpath-colab", "pull"], check=True)
import importlib, mcpath.ui as mcui
importlib.reload(mcui)
from mcpath.ui import launch
launch(
    defaults_url="https://raw.githubusercontent.com/enesemretas/mcpath-colab/main/config/defaults.yaml",
    show_title="MCPath (Python readpdb)"
)



TypeError: _list_or_custom() got an unexpected keyword argument 'max'